In [1]:
import torch

# tensor[tensor]

In [2]:
a = torch.tensor([1, 2, 3, 4])
ind = torch.tensor([3, 2, 1])

In [3]:
a[ind]

tensor([4, 3, 2])

# mean的输出

In [4]:
torch.tensor(2)

tensor(2)

In [6]:
torch.tensor([1, 2, 3]).float().mean()

tensor(2.)

# 格式化输出

In [15]:
print("{:6.2f} 1".format(5.3))
print("{:6.2f}".format(555555))
print("{:6.2f}".format(56555.31))

  5.30 1
555555.00
56555.31


6：占据6个字符，.2：小数点后2位

# leaf grad测试

In [13]:
leaf = torch.tensor(1.0, requires_grad = True)
loss = 2 * leaf

In [6]:
leaf.grad is None

True

In [8]:
leaf.grad.zero_() # if not None 不可少

AttributeError: 'NoneType' object has no attribute 'zero_'

In [9]:
loss.grad # 非叶子不留grad，所以不可w -= w.grad,那样叶子没了

/tmp/ipykernel_2745/3355736690.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995026/work/build/aten/src/ATen/core/TensorBody.h:486.)
  loss.grad


In [10]:
loss.backward()

In [11]:
leaf.grad

tensor(2.)

In [12]:
loss.backward() # intermediate values of the graph are freed when you call .backward(),一次前向 -> 一次后向

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [27]:
leaf = torch.tensor(1.0, requires_grad = True)
loss = 2 * leaf
f1 = 3 * loss
f2 = 4 * loss
f1.backward()
leaf.grad

tensor(6.)

In [28]:
f2.backward() # 公用分支，也只能back一次！

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

# SGD([paras])

In [2]:
import torch
import torch.optim as optim
import torch.nn as nn

In [4]:
model = nn.Linear(2, 1)
model.parameters()

<generator object Module.parameters at 0x7fcd5208ef10>

In [5]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.6780, -0.4766]], requires_grad=True),
 Parameter containing:
 tensor([-0.3371], requires_grad=True)]

In [10]:
paras = torch.tensor(1)
opt = optim.SGD([paras], 0.1) # []: "an iterable of Tensors or dicts"
opt

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

# nn input

In [19]:
model = nn.Linear(1, 1)
d_1 = torch.tensor([2.0])
d_2 = torch.tensor([[2.0]])
y = model(d_1) # “侥幸成功”
y

tensor([2.0619], grad_fn=<AddBackward0>)

In [20]:
y = model(d_2) # correct
y

tensor([[2.0619]], grad_fn=<AddmmBackward0>)

In [21]:
d_0 = torch.tensor(2.0)
y = model(d_0)

RuntimeError: both arguments to matmul need to be at least 1D, but they are 0D and 2D